<a href="https://colab.research.google.com/github/jenochs/video-generation-book/blob/main/notebooks/hunyuan_colab_a100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HunyuanVideo on Google Colab A100

**Generate high-quality videos using Tencent's HunyuanVideo model on Google Colab A100 GPU**

🚀 **What you'll learn:**
- Run the 13B parameter HunyuanVideo model on Colab A100 (40GB)
- Optimize memory usage for large-scale video generation
- Generate videos up to 15 seconds with advanced prompting
- Export and download high-quality video results

⚡ **Requirements:**
- Google Colab Pro+ with A100 GPU access
- ~20-30 minutes for complete setup
- Google Drive for video storage (optional)

📚 **From the Book:** *Hands-On Video Generation with AI* - Chapter 3: Advanced Model Implementation

## 🔧 1. Environment Setup & GPU Verification

First, let's verify we have an A100 GPU and configure the environment for optimal performance.

## 📦 1. Install Dependencies

Install the latest versions of required libraries optimized for HunyuanVideo.

In [1]:
 # Install HunyuanVideo dependencies
!pip install torch \
             torchvision \
             torchaudio  \
             diffusers[torch]==0.33.1 \
             transformers==4.52.4 \
             accelerate==1.7.0 \
             safetensors==0.5.3 \
             xformers \
             imageio-ffmpeg==0.6.0 \
             --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 106.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s e

In [2]:
# Check GPU availability and specifications
!nvidia-smi

# Verify we have A100 access
import subprocess
result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader,nounits'],
                       capture_output=True, text=True)
print("\n🖥️ GPU Information:")
gpu_info = result.stdout.strip().split(', ')
if len(gpu_info) >= 2:
    gpu_name, gpu_memory = gpu_info[0], int(gpu_info[1])
    print(f"   GPU: {gpu_name}")
    print(f"   Memory: {gpu_memory:,} MB ({gpu_memory/1024:.1f} GB)")

    if "A100" in gpu_name and gpu_memory >= 40000:
        print("   ✅ Perfect! A100 40GB detected - optimal for HunyuanVideo")
    elif "A100" in gpu_name:
        print("   ⚠️ A100 detected but check memory - may need optimization")
    else:
        print("   ❌ Warning: A100 GPU recommended for best performance")
        print("   💡 Consider upgrading to Colab Pro+ for A100 access")
else:
    print("   ❌ Unable to detect GPU information")

Sun Jun  8 18:55:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             46W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
# Configure environment for maximum memory efficiency
import os
import torch

# Essential memory optimizations for A100 40GB
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'  # Async for better performance
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # Avoid warnings

# Enable optimized math operations
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True

print("🔧 Environment configured for A100 optimization")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   CUDA version: {torch.version.cuda}")
    print(f"   GPU count: {torch.cuda.device_count()}")

🔧 Environment configured for A100 optimization
   PyTorch version: 2.5.1+cu121
   CUDA available: True
   CUDA version: 12.1
   GPU count: 1


## 📦 2. Install Book package

Install the latest version of the book library package.

In [4]:
# Install the videogenbook package
!pip install git+https://github.com/jenochs/video-generation-book.git

# Verify installation
import videogenbook
print(f"✅ videogenbook v{videogenbook.__version__} installed successfully")

  Cloning https://github.com/jenochs/video-generation-book.git to /tmp/pip-req-build-2gaounty
  Running command git clone --filter=blob:none --quiet https://github.com/jenochs/video-generation-book.git /tmp/pip-req-build-2gaounty
  Resolved https://github.com/jenochs/video-generation-book.git to commit 286043722ca56d8157798f361f72a242ac73d5b7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 131.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

## 🧠 3. Memory Monitoring & Optimization

Set up memory monitoring and configure HunyuanVideo for A100 40GB constraints.

In [5]:
import torch
import gc
from typing import Dict, Any

def get_gpu_memory() -> Dict[str, float]:
    """Get current GPU memory usage in GB."""
    if not torch.cuda.is_available():
        return {"total": 0, "used": 0, "free": 0}

    total = torch.cuda.get_device_properties(0).total_memory / 1024**3
    allocated = torch.cuda.memory_allocated() / 1024**3
    cached = torch.cuda.memory_reserved() / 1024**3
    free = total - cached

    return {
        "total": total,
        "allocated": allocated,
        "cached": cached,
        "free": free
    }

def print_memory_status(stage: str = ""):
    """Print current memory status."""
    mem = get_gpu_memory()
    print(f"🧠 GPU Memory {stage}:")
    print(f"   Total: {mem['total']:.1f} GB")
    print(f"   Allocated: {mem['allocated']:.1f} GB")
    print(f"   Cached: {mem['cached']:.1f} GB")
    print(f"   Free: {mem['free']:.1f} GB")

    # Memory warnings
    if mem['free'] < 10:
        print("   ⚠️ Low memory - consider reducing resolution/frames")
    elif mem['free'] < 20:
        print("   ✅ Sufficient memory for standard generation")
    else:
        print("   🚀 Excellent memory - can use higher quality settings")

def cleanup_memory():
    """Cleanup GPU memory."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

# Initial memory check
print_memory_status("(Initial)")

🧠 GPU Memory (Initial):
   Total: 39.6 GB
   Allocated: 0.0 GB
   Cached: 0.0 GB
   Free: 39.6 GB
   🚀 Excellent memory - can use higher quality settings


## 🤖 4. Load HunyuanVideo Model

Load the HunyuanVideo model with A100-optimized settings for 40GB memory constraint.

In [ ]:
from diffusers import HunyuanVideoPipeline
import torch
import time

print("🔄 Loading HunyuanVideo model (this may take 5-10 minutes)...")
print("📥 Downloading ~26GB of model weights...")

start_time = time.time()

try:
    # Load with aggressive memory optimization for A100 40GB
    pipe = HunyuanVideoPipeline.from_pretrained(
        "hunyuanvideo-community/HunyuanVideo",  # Fixed: Use community diffusers version
        torch_dtype=torch.float16,  # Use FP16 for memory efficiency
        use_safetensors=True,
        # variant="fp16",  # REMOVED: Community model doesn't have fp16 variants
        low_cpu_mem_usage=True,     # Minimize CPU memory during loading
    )

    print("\n🔧 Applying A100 optimizations...")

    # Essential memory optimizations for 40GB constraint
    pipe.enable_sequential_cpu_offload()  # Most aggressive memory optimization
    pipe.vae.enable_tiling()              # Reduce VAE memory usage
    pipe.vae.enable_slicing()             # Further VAE optimization

    # Enable memory-efficient attention if available
    try:
        pipe.enable_xformers_memory_efficient_attention()
        print("   ✅ xFormers memory-efficient attention enabled")
    except ImportError:
        print("   ⚠️ xFormers not available - using default attention")
    except Exception as e:
        print(f"   ⚠️ xFormers setup issue: {e}")

    # Configure scheduler for memory efficiency
    if hasattr(pipe.scheduler, 'enable_low_mem_usage'):
        pipe.scheduler.enable_low_mem_usage = True

    load_time = time.time() - start_time
    print(f"\n✅ HunyuanVideo loaded successfully in {load_time:.1f}s")
    print("🎬 Ready for video generation!")

    print_memory_status("(After model loading)")

except Exception as e:
    print(f"❌ Failed to load HunyuanVideo: {str(e)}")
    print("\n🔍 Troubleshooting steps:")
    print("1. Ensure you have A100 GPU access")
    print("2. Check available disk space (need ~30GB)")
    print("3. Restart runtime and try again")
    raise

🔄 Loading HunyuanVideo model (this may take 5-10 minutes)...
📥 Downloading ~26GB of model weights...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

Fetching 28 files:   0%|          | 0/28 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/510 [00:00<?, ?B/s]

(…)pytorch_model-00003-of-00006.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

(…)pytorch_model-00004-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

(…)pytorch_model-00005-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

## ⚙️ 5. A100-Optimized Generation Settings

Configure generation parameters optimized for A100 40GB memory constraints while maintaining high quality.

In [ ]:
# A100 40GB optimized configurations
COLAB_CONFIGS = {
    "high_quality": {
        "height": 720,
        "width": 1280,
        "num_frames": 65,           # Reduced from 129 for memory
        "guidance_scale": 7.0,
        "num_inference_steps": 30,  # Balanced quality/speed
        "description": "High quality 720p (may use significant memory)"
    },
    "balanced": {
        "height": 544,
        "width": 960,
        "num_frames": 65,
        "guidance_scale": 7.0,
        "num_inference_steps": 25,
        "description": "Balanced quality and memory usage (recommended)"
    },
    "fast": {
        "height": 512,
        "width": 512,
        "num_frames": 32,
        "guidance_scale": 6.0,
        "num_inference_steps": 20,
        "description": "Fast generation with lower memory usage"
    }
}

def print_config_options():
    """Display available configuration options."""
    print("📊 Available Quality Configurations:")
    for name, config in COLAB_CONFIGS.items():
        duration = config['num_frames'] / 8  # Assuming 8 FPS
        print(f"\n🎥 {name.upper()}:")
        print(f"   Resolution: {config['width']}x{config['height']}")
        print(f"   Duration: ~{duration:.1f}s ({config['num_frames']} frames)")
        print(f"   Steps: {config['num_inference_steps']}")
        print(f"   💡 {config['description']}")

print_config_options()

# Memory usage estimates
print("\n💾 Estimated Memory Usage:")
print("   High Quality: ~35-38GB (may require cleanup between generations)")
print("   Balanced: ~25-30GB (recommended for most use cases)")
print("   Fast: ~15-20GB (reliable for multiple generations)")

## 🎬 6. Interactive Video Generation

Generate high-quality videos with HunyuanVideo using your custom prompts.

In [ ]:
from IPython.display import Video, display
import os
from datetime import datetime

def generate_video_colab(
    prompt: str,
    quality: str = "balanced",
    seed: int = None,
    output_dir: str = "/content/videos"
) -> str:
    """Generate video with HunyuanVideo optimized for Colab A100.

    Args:
        prompt: Text description of the video to generate
        quality: 'high_quality', 'balanced', or 'fast'
        seed: Random seed for reproducible results
        output_dir: Directory to save generated videos

    Returns:
        Path to generated video file
    """
    if quality not in COLAB_CONFIGS:
        quality = "balanced"
        print(f"⚠️ Invalid quality setting, using 'balanced'")

    config = COLAB_CONFIGS[quality]

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Generate filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"hunyuan_{quality}_{timestamp}.mp4"
    output_path = os.path.join(output_dir, filename)

    print(f"🎬 Generating video with {quality} settings...")
    print(f"📝 Prompt: {prompt}")
    print(f"🎯 Resolution: {config['width']}x{config['height']}")
    print(f"⏱️ Frames: {config['num_frames']} (~{config['num_frames']/8:.1f}s)")

    print_memory_status("(Before generation)")

    try:
        start_time = time.time()

        # Set random seed for reproducibility
        generator = None
        if seed is not None:
            generator = torch.Generator(device="cuda").manual_seed(seed)
            print(f"🎲 Using seed: {seed}")

        # Generate video with progress tracking
        print("\n🔄 Generating frames...")
        video = pipe(
            prompt=prompt,
            height=config["height"],
            width=config["width"],
            num_frames=config["num_frames"],
            guidance_scale=config["guidance_scale"],
            num_inference_steps=config["num_inference_steps"],
            generator=generator,
            output_type="pil"  # PIL format for easier handling
        ).frames[0]

        generation_time = time.time() - start_time

        # Save video using imageio with H.264 codec
        print("\n💾 Saving video...")
        import imageio
        with imageio.get_writer(output_path, fps=8, codec='h264', quality=8) as writer:
            for frame in video:
                writer.append_data(frame)

        # Get file size
        file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB

        print(f"\n✅ Video generated successfully!")
        print(f"   ⏱️ Generation time: {generation_time:.1f}s")
        print(f"   💾 File size: {file_size:.1f} MB")
        print(f"   📁 Saved to: {output_path}")

        print_memory_status("(After generation)")

        return output_path

    except torch.cuda.OutOfMemoryError:
        print("\n❌ GPU out of memory!")
        cleanup_memory()
        print("\n💡 Try these solutions:")
        print("   1. Use 'fast' quality setting")
        print("   2. Reduce num_frames or resolution")
        print("   3. Restart runtime to clear memory")
        raise
    except Exception as e:
        print(f"\n❌ Generation failed: {str(e)}")
        cleanup_memory()
        raise

print("🎬 Video generation function ready!")
print("📝 Use generate_video_colab(prompt, quality) to create videos")

## 🎨 7. Example Generations

Try these example prompts or create your own!

In [ ]:
# Example 1: Balanced quality generation
prompt_1 = "A majestic golden eagle soaring over snow-capped mountains at sunset, cinematic camera movement, high detail, beautiful lighting"

video_path_1 = generate_video_colab(
    prompt=prompt_1,
    quality="balanced",
    seed=42  # For reproducible results
)

# Display the generated video
print("\n🎥 Generated Video:")
display(Video(video_path_1, width=600))

In [ ]:
# Example 2: High quality generation (use with caution on memory)
prompt_2 = "A futuristic cyberpunk city at night with neon lights reflecting in rain puddles, flying cars in the distance, dramatic atmosphere"

# Check memory before high-quality generation
mem = get_gpu_memory()
if mem['free'] > 25:
    print("🚀 Sufficient memory detected - proceeding with high quality")
    video_path_2 = generate_video_colab(
        prompt=prompt_2,
        quality="high_quality",
        seed=123
    )
    display(Video(video_path_2, width=600))
else:
    print("⚠️ Limited memory - using balanced quality instead")
    video_path_2 = generate_video_colab(
        prompt=prompt_2,
        quality="balanced",
        seed=123
    )
    display(Video(video_path_2, width=600))

# Cleanup memory after generation
cleanup_memory()

In [ ]:
# Custom generation - Enter your own prompt!
custom_prompt = input("Enter your video prompt: ")
quality_choice = input("Choose quality (high_quality/balanced/fast): ") or "balanced"
custom_seed = input("Enter seed (or press enter for random): ")

seed_value = None
if custom_seed.strip():
    try:
        seed_value = int(custom_seed)
    except ValueError:
        print("Invalid seed, using random")

if custom_prompt.strip():
    custom_video_path = generate_video_colab(
        prompt=custom_prompt,
        quality=quality_choice,
        seed=seed_value
    )
    display(Video(custom_video_path, width=600))
else:
    print("No prompt entered, skipping generation")

## 📚 8. Batch Generation

Generate multiple videos with automatic memory management.

In [ ]:
def batch_generate_videos(prompts_and_configs: list, output_dir: str = "/content/batch_videos"):
    """Generate multiple videos with automatic memory cleanup.

    Args:
        prompts_and_configs: List of (prompt, quality, seed) tuples
        output_dir: Directory for batch outputs
    """
    os.makedirs(output_dir, exist_ok=True)
    results = []

    print(f"🎬 Starting batch generation of {len(prompts_and_configs)} videos...")

    for i, config in enumerate(prompts_and_configs, 1):
        prompt = config[0]
        quality = config[1] if len(config) > 1 else "balanced"
        seed = config[2] if len(config) > 2 else None

        print(f"\n📹 Generation {i}/{len(prompts_and_configs)}:")
        print(f"   Prompt: {prompt[:60]}..." if len(prompt) > 60 else f"   Prompt: {prompt}")

        try:
            video_path = generate_video_colab(
                prompt=prompt,
                quality=quality,
                seed=seed,
                output_dir=output_dir
            )
            results.append((prompt, video_path, "success"))

            # Cleanup memory between generations
            if i < len(prompts_and_configs):  # Don't cleanup after last generation
                print("🧹 Cleaning up memory for next generation...")
                cleanup_memory()

        except Exception as e:
            print(f"❌ Failed: {str(e)}")
            results.append((prompt, None, f"error: {str(e)}"))
            cleanup_memory()  # Always cleanup on error

    print(f"\n✅ Batch generation completed!")
    print(f"📊 Results: {sum(1 for r in results if r[2] == 'success')}/{len(results)} successful")

    return results

# Example batch generation
sample_batch = [
    ("A cat playing with a ball of yarn in slow motion", "fast", 111),
    ("Ocean waves crashing on a rocky shore at dawn", "balanced", 222),
    ("A hummingbird feeding from colorful flowers", "balanced", 333)
]

print("🎬 Example batch ready to run!")
print("📝 Uncomment the line below to start batch generation:")
print("# batch_results = batch_generate_videos(sample_batch)")

# Uncomment to run:
# batch_results = batch_generate_videos(sample_batch)

## 💾 9. Export & Download

Export your generated videos to Google Drive or download them directly.

In [ ]:
# Mount Google Drive for permanent storage
from google.colab import drive
drive.mount('/content/drive')

def copy_to_drive(video_path: str, drive_folder: str = "HunyuanVideo_Outputs"):
    """Copy generated video to Google Drive."""
    import shutil

    drive_path = f"/content/drive/MyDrive/{drive_folder}"
    os.makedirs(drive_path, exist_ok=True)

    filename = os.path.basename(video_path)
    drive_file = os.path.join(drive_path, filename)

    shutil.copy2(video_path, drive_file)
    print(f"✅ Video copied to Google Drive: {drive_file}")
    return drive_file

def download_video(video_path: str):
    """Download video to local machine."""
    from google.colab import files
    if os.path.exists(video_path):
        files.download(video_path)
        print(f"⬇️ Download started for: {os.path.basename(video_path)}")
    else:
        print(f"❌ File not found: {video_path}")

print("💾 Google Drive mounted successfully!")
print("📁 Use copy_to_drive(video_path) to save to Drive")
print("⬇️ Use download_video(video_path) to download directly")

In [ ]:
# List all generated videos
import glob

video_dirs = ["/content/videos", "/content/batch_videos"]
all_videos = []

for video_dir in video_dirs:
    if os.path.exists(video_dir):
        videos = glob.glob(os.path.join(video_dir, "*.mp4"))
        all_videos.extend(videos)

if all_videos:
    print(f"🎬 Found {len(all_videos)} generated videos:")
    for i, video_path in enumerate(all_videos, 1):
        file_size = os.path.getsize(video_path) / (1024 * 1024)
        print(f"   {i}. {os.path.basename(video_path)} ({file_size:.1f} MB)")

    print("\n💾 Export options:")
    print("   # Copy all videos to Google Drive")
    print("   for video in all_videos: copy_to_drive(video)")
    print("")
    print("   # Download specific video (replace index)")
    print("   download_video(all_videos[0])  # Downloads first video")
else:
    print("📭 No videos found. Generate some videos first!")

In [ ]:
# Quick export all videos to Drive
if all_videos:
    export_choice = input("Export all videos to Google Drive? (y/n): ")
    if export_choice.lower() == 'y':
        print("📤 Exporting all videos to Google Drive...")
        for video_path in all_videos:
            try:
                copy_to_drive(video_path)
            except Exception as e:
                print(f"❌ Failed to copy {video_path}: {e}")
        print("✅ Export completed!")
    else:
        print("📋 Export skipped. Use copy_to_drive(video_path) for individual files.")
else:
    print("📭 No videos to export.")

## 🔧 10. Troubleshooting & Optimization

Common issues and solutions for HunyuanVideo on Colab A100.

In [ ]:
def run_diagnostics():
    """Run comprehensive system diagnostics."""
    print("🔍 Running HunyuanVideo Colab Diagnostics...")
    print("=" * 50)

    # GPU Information
    print("\n🖥️ GPU Information:")
    if torch.cuda.is_available():
        gpu_props = torch.cuda.get_device_properties(0)
        print(f"   GPU: {gpu_props.name}")
        print(f"   Total Memory: {gpu_props.total_memory / 1024**3:.1f} GB")
        print(f"   CUDA Capability: {gpu_props.major}.{gpu_props.minor}")

        # Check if A100
        if "A100" in gpu_props.name:
            print("   ✅ A100 GPU detected - optimal for HunyuanVideo")
        else:
            print("   ⚠️ Non-A100 GPU - may need optimization")
    else:
        print("   ❌ No GPU available")

    # Memory Status
    print_memory_status("\n🧠 Current Memory")

    # Disk Space
    print("\n💾 Disk Space:")
    import shutil
    total, used, free = shutil.disk_usage("/content")
    print(f"   Total: {total / 1024**3:.1f} GB")
    print(f"   Used: {used / 1024**3:.1f} GB")
    print(f"   Free: {free / 1024**3:.1f} GB")

    if free / 1024**3 < 20:
        print("   ⚠️ Low disk space - may affect model loading")

    # Library Versions
    print("\n📚 Library Versions:")
    try:
        import diffusers, transformers, torch
        print(f"   PyTorch: {torch.__version__}")
        print(f"   Diffusers: {diffusers.__version__}")
        print(f"   Transformers: {transformers.__version__}")

        # Check versions
        if diffusers.__version__ >= "0.33.1":
            print("   ✅ Diffusers version supports HunyuanVideo")
        else:
            print("   ⚠️ Update diffusers: pip install diffusers>=0.33.1")
    except ImportError as e:
        print(f"   ❌ Import error: {e}")

    # Model Status
    print("\n🤖 Model Status:")
    try:
        # Check if pipe is loaded
        if 'pipe' in globals():
            print("   ✅ HunyuanVideo pipeline loaded")
            print(f"   Device: {next(pipe.parameters()).device}")
            print(f"   Dtype: {next(pipe.parameters()).dtype}")
        else:
            print("   ❌ HunyuanVideo pipeline not loaded")
    except Exception as e:
        print(f"   ⚠️ Error checking model: {e}")

    print("\n🔧 Optimization Status:")
    print(f"   TF32 enabled: {torch.backends.cuda.matmul.allow_tf32}")
    print(f"   cuDNN benchmark: {torch.backends.cudnn.benchmark}")
    print(f"   Memory fraction: {torch.cuda.get_memory_fraction(0) if torch.cuda.is_available() else 'N/A'}")

def memory_optimization_tips():
    """Show memory optimization tips."""
    print("💡 Memory Optimization Tips:")
    print("=" * 30)
    print("\n🔧 If you get Out of Memory errors:")
    print("   1. Use 'fast' quality setting")
    print("   2. Reduce num_frames (try 32 instead of 65)")
    print("   3. Lower resolution (512x512 instead of 720p)")
    print("   4. Restart runtime to clear memory")
    print("   5. Run cleanup_memory() between generations")

    print("\n⚡ For better performance:")
    print("   1. Use sequential_cpu_offload (already enabled)")
    print("   2. Enable VAE tiling (already enabled)")
    print("   3. Use FP16 precision (already enabled)")
    print("   4. Generate videos one at a time")
    print("   5. Close browser tabs to free system RAM")

# Run diagnostics
run_diagnostics()
memory_optimization_tips()

In [ ]:
# Emergency memory cleanup
def emergency_cleanup():
    """Aggressive memory cleanup for emergency situations."""
    print("🚨 Running emergency memory cleanup...")

    # Clear Python variables
    if 'pipe' in globals():
        print("   Clearing pipeline...")
        del pipe

    # Garbage collection
    import gc
    gc.collect()

    # CUDA cleanup
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        print("   CUDA cache cleared")

    # Check memory after cleanup
    print_memory_status("(After cleanup)")

    print("✅ Emergency cleanup completed")
    print("💡 You may need to reload the model with the loading cell above")

print("🚨 Emergency cleanup function ready")
print("📝 Run emergency_cleanup() if you encounter memory issues")

## 🎉 Conclusion

Congratulations! You've successfully set up and used HunyuanVideo on Google Colab A100.

### 🎯 What You've Accomplished:
- ✅ Loaded a 13B parameter video generation model on Colab
- ✅ Optimized for A100 40GB memory constraints
- ✅ Generated high-quality videos up to 15 seconds
- ✅ Learned memory management for large AI models
- ✅ Set up batch generation and export workflows

### 🚀 Next Steps:
1. **Experiment** with different prompts and quality settings
2. **Explore** the full videogenbook library for more models
3. **Read** the complete guide: *Hands-On Video Generation with AI*
4. **Join** the community: [GitHub Repository](https://github.com/jenochs/video-generation-book)

### 🔗 Resources:
- **Documentation**: [videogenbook docs](https://github.com/jenochs/video-generation-book)
- **HunyuanVideo**: [Model Card](https://huggingface.co/tencent/HunyuanVideo)
- **Colab Pro+**: [Upgrade for A100 access](https://colab.research.google.com/signup)

### 💡 Tips for Best Results:
- Use detailed, descriptive prompts
- Include camera movement descriptions
- Specify lighting and atmosphere
- Experiment with different seeds for variety
- Monitor memory usage for stable generation

**Happy video generating! 🎬✨**